## Carregando e entendendo a base de dados

Inicialmente iremos carregar os dados e verificar se existe a necessidade de fazer algum tipo de 
pre-processamento, e para isso utilizaremos a biblioteca `Pandas` que contém um conjunto de funções
para se trabalhar com dados tabulares que é o nosso caso.

In [129]:
import pandas as pd

# nomes de cada uma das colunas segundo a documentação da base.
attribute_names = [
  'Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
  'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
  'Normal Nucleoli', 'Mitoses', 'Class'
]

url = 'https://raw.githubusercontent.com/joaofelipesus/overfit-underfit-hyperparams/main/breast-cancer-wisconsin.data'

dataset = pd.read_csv(url, names=attribute_names)

dataset

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


## Pre-processamento

Com a base carregada podemos verificar se existe algum registro incompleto, na documentação da base
é dito que existem dezesseis exemplares em que o atributo "Bare Nuclei" possui valor `?`, indicando 
que este valor não deve ser levado em consideração. Existem algumas abordagens para tratar este tipo 
de cenário como adicionar o valor médio ou mediano do atributo porém iremos descartar estes exemplares.

Para concluir a etapa de inicial iremos remover a coluna "Sample code number" uma vez que seu valor 
não é relacionado a uma característica e utilizá-lo nas etapas de treinamento e classificação não faz 
sentido. Por fim iremos modificar os valores referentes as classes, que originalmente estão 
distribuídos entre 2 (benigno) e 4 (maligno), iremos substituir 2 por 0 e 4 por 1 dessa forma fica 
mais simples entender a classe do registro.

In [130]:
# Remove exemplarem que possuem valor '?' atribuido ao atributo "Bare Nuclei"
dataset = dataset[dataset["Bare Nuclei"] != '?']

# Remove a coluna "Sample code number"
dataset = dataset.drop(columns=['Sample code number'])

# Substitui valores das classes de 2 e 4 para 0 e 1
dataset.loc[dataset['Class'] == 2, 'Class'] = 0
dataset.loc[dataset['Class'] == 4, 'Class'] = 1

dataset



,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,0
1,5,4,4,5,7,10,3,2,1,0
2,3,1,1,1,2,2,3,1,1,0
3,6,8,8,1,3,4,3,7,1,0
4,4,1,1,3,2,1,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...
694,3,1,1,1,3,2,1,1,1,0
695,2,1,1,1,2,1,1,1,1,0
696,5,10,10,3,7,3,8,10,2,1
697,4,8,6,4,3,4,10,6,1,1


## Overfit


In [149]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import numpy as np

SEED = 42
np.random.seed(SEED)

# Isola apenas as colunas contendo atributos da coluna contendo as classes.
X = dataset.iloc[:, 0:9]
y = dataset.Class

# Comentar atributos
# Divide a base em dois grupos, treino e teste
X_train, X_test, y_train, y_test = train_test_split(
  X,
  y,
  test_size=0.2,
  random_state=SEED,
  stratify=y
)

classifier = DecisionTreeClassifier(
  random_state=SEED,
  max_depth=1,
  min_samples_split=2
)

classifier.fit(X_train, y_train)

classifier.score(X_test, y_test)

# X_train.shape

# print(y_test.value_counts())

# print(y_train.value_counts())



0.8978102189781022